In [1]:
import autograd.numpy as np
import scipy as sp
from scipy import optimize

import VariationalBayes as vb
from  VariationalBayes import ExponentialFamilies as ef
from  VariationalBayes.SparseObjectives import Objective

import matplotlib.pyplot as plt
%matplotlib inline


In [249]:
params = vb.ModelParamsDict('par')
params.push_param(vb.VectorParam('beta', size=4))

params['beta'].set(np.random.random(params['beta'].size()))

class Model(object):
    def __init__(self, params):
        self.params = params
        self.data = np.linspace(1, 2, self.params['beta'].size())
        self.data = self.data / np.linalg.norm(self.data)
        self.lam = np.array(0.)
        self.mu = np.array(1.)
        self.objective = Objective(self.params, self.get_objective)
        self.constraint = Objective(self.params, self.get_constraint_vec)

    def log_lik(self):
        normalize = False
        beta = self.params['beta'].get()
        if normalize:
            beta /= np.linalg.norm(beta)

        return -1 * np.dot(beta, self.data)
    
    def get_beta_norm(self):
        return np.linalg.norm(self.params['beta'].get())
    
    def get_constraint_vec(self):
        return np.array([ self.get_beta_norm() - 1])
    
    def dual_term(self):
        return -1 * np.dot(self.lam, self.get_constraint_vec())
    
    def constraint_term(self):
        return 0.5 * np.dot(self.mu, self.get_constraint_vec() ** 2)
        
    def get_objective(self):
        return np.squeeze(self.log_lik() + self.constraint_term() + self.dual_term())
    
    def optimize(self, x0=None, maxiter=100, print_every=1, verbose=False):
        self.objective.logger.print_every = print_every
        self.objective.logger.initialize()
        if x0 is None:
            x0 = self.params.get_free()
        
        vb_opt = optimize.minimize(
            lambda par: self.objective.fun_free(par, verbose=verbose),
            x0 = x0,
            jac = self.objective.fun_free_grad,
            hessp = self.objective.fun_free_hvp,
            method = 'trust-ncg',
            options = {'maxiter': maxiter, 'gtol': 1e-6, 'disp': verbose})
        self.params.set_free(vb_opt.x)
        return vb_opt

    def update_lambda(self):
        new_lam = self.lam - self.get_constraint_vec() * self.mu#
        self.lam = new_lam
        print('Beta norm: ', self.get_beta_norm(),
              ' lambda: ', self.lam,
              ' constraint: ', self.get_constraint_vec())


model = Model(params)
print(model.get_objective())
vb_opt = model.optimize()

print(model.get_beta_norm())
print(model.constraint.fun_free_grad(vb_opt.x))
print(model.get_constraint_vec() * model.mu)
print('Beta norm: ', model.get_beta_norm(), ' lambda: ', model.lam)


-0.6173231176357886
2.00000007679
[ 0.32349837  0.43133117  0.53916388  0.64699655]
[ 1.00000008]
Beta norm:  2.00000007679  lambda:  0.0


In [265]:
model.mu = 2.0
model.lam = 0.0
model.params['beta'].set(np.full(model.params['beta'].size(), 0.1))
model.get_objective()

array(0.44590100823381096)

In [270]:
model.optimize()
model.update_lambda()
print(model.params['beta'].get() / model.data)


Beta norm:  0.999999999847  lambda:  [-1.]  constraint:  [ -1.52752144e-10]
[ 1.  1.  1.  1.]
